In [3]:
import torch
import os
from PIL import Image
import torchvision.transforms as T
import pandas as pd
from rle import rle_decode
import numpy as np
from tqdm import tqdm

torch_ver_major = int(torch.__version__.split('.')[0])
dtype_index = torch.int32 if torch_ver_major >= 2 else torch.long

train_dir = "/kaggle/input/blood-vessel-segmentation/train/"
msks_dir = f"{train_dir}kidney_1_dense/labels/"
imgs_dir = f"{train_dir}kidney_1_dense/images/"
slices_ids = sorted(os.listdir(imgs_dir))
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
w = Image.open(msks_dir + slices_ids[0]).width
h = Image.open(msks_dir + slices_ids[0]).height
print(w, h)

912 1303


In [5]:
class TestMetricDataset(torch.utils.data.Dataset):
    def __init__(self, sub_df, msks_dir, slices_ids, transform=None, target_transform=None):
        self.sub_df = sub_df
        self.msks_dir = msks_dir
        self.slices_ids = slices_ids
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.slices_ids)
    
    def __getitem__(self, idx):
        slice_id = self.slices_ids[idx]

        pred_rle = sub_df.iloc[idx]["rle"]
        pred = rle_decode(pred_rle, (h, w))
        pred = torch.from_numpy(pred)

        target_path = self.msks_dir + slice_id 
        target = Image.open(target_path)

        if self.target_transform is not None:
            target = self.target_transform(target).squeeze()

        return pred, target

In [6]:
sub_df = pd.read_csv("ref_sub.csv")
bs = 5

target_transform = T.Compose([
    T.PILToTensor(), 
])

ds = TestMetricDataset(sub_df=sub_df, msks_dir=msks_dir, slices_ids=slices_ids, target_transform=target_transform)
dl = torch.utils.data.DataLoader(ds, batch_size=bs)

In [7]:
pred, target = next(iter(dl))
print(pred.shape, target.shape)

torch.Size([5, 1303, 912]) torch.Size([5, 1303, 912])


In [8]:
from surface_dice import SurfaceDiceMetric
metric = SurfaceDiceMetric(10)

In [9]:
pred = torch.vstack([torch.zeros((1, h, w), dtype=torch.uint8), pred])
target = torch.vstack([torch.zeros((1, h, w), dtype=torch.uint8), target])
print(pred.shape, target.shape)

torch.Size([6, 1303, 912]) torch.Size([6, 1303, 912])


In [10]:
unfold = torch.nn.Unfold(kernel_size=(2, 2), padding=1)
# compute the surface area for pred
surface_pred = pred.to(torch.float16).view(3, 2, h, w)
print("suraface_pred shape:", surface_pred.shape)
cubes_float = unfold(surface_pred).permute(1, 0, 2).reshape(8, -1)
print("cubes_float shape:", cubes_float.shape)
cubes_byte = torch.zeros(cubes_float.size(1), dtype=dtype_index)
print("cubes_byte shape:", cubes_byte.shape)

suraface_pred shape: torch.Size([3, 2, 1303, 912])
cubes_float shape: torch.Size([8, 3571656])
cubes_byte shape: torch.Size([3571656])


In [ ]:
# x = torch.arange(3*8*5).view(3, 8, 5)
# print(x)
# print(x.shape)
# print(x.permute(1, 0, 2).reshape(8, -1))


In [11]:
for k in range(8):
    cubes_byte += cubes_float[k, :].to(dtype_index) << k

In [14]:
cubes_area = metric.area[cubes_byte]
cubes_area